In [1]:
import numpy as np
import pandas as pd

import json
import glob
import os
import re

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
base = "../exp/"
paths = glob.glob(base + "rand/*.json") + glob.glob(base + "rand_live/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
queries = pd.DataFrame.from_records([{
    **algo,
    'potential': run['potential'],
    'graph': run['args'][1],
    'metric': 'predicted' if run['program'] in ['predicted_queries', 'predicted_queries_no_topo'] else run['args'][3],
    'topo': 'no_topo' not in run['program']
} for run in data for algo in run['algo_runs']])

queries['num_queue_pops_k'] = queries['num_queue_pops'] / 1000
queries['increase_percent'] = (queries['result'] / queries['lower_bound'] - 1) * 100

In [4]:
runtime_pattern = re.compile(".*running time : (\\d+)musec.*")

def parse_flowcutter_partition_output(path):
    stats = { 'cch_ordering_running_time_s': 0.0 }

    with open(path, 'r') as f:
        for line in f:
            if not 'graph' in stats:
                stats['graph'] = line.strip()
            else:
                match = runtime_pattern.match(line)
                if match:
                    stats['cch_ordering_running_time_s'] += int(match[1]) / 1000000

    return stats

In [5]:
cch_ordering = pd.DataFrame.from_records([parse_flowcutter_partition_output(path) for path in glob.glob(base + "preprocessing/*.out")])

In [6]:
data = [json.load(open(path)) for path in glob.glob(base + "preprocessing/*.json")]
prepros = pd.DataFrame.from_records([{
    **run,
    'potential': { 'cchpot_pre': 'lower_bound_cch_pot', 'multi_metric_pre': 'multi_metric_pot', 'interval_min_pre': 'interval_min_pot' }[run['program']],
    'graph': run['args'][1],
} for run in data])

In [7]:
data = [json.load(open(path)) for path in glob.glob(base + "customization/*.json")]
customs = pd.DataFrame.from_records([{
    **run,
    'potential': { 'multi_metric_live_customization': 'multi_metric_pot', 'interval_min_live_customization': 'interval_min_pot' }[run['program']],
    'graph': run['args'][1],
    'metric': run['args'][3],
} for run in data])

In [8]:
queries.query('topo').groupby(['graph', 'metric', 'potential']).mean()[['running_time_ms', 'num_queue_pops_k', 'increase_percent']]

running_time_ms  \
graph                                              metric          potential                              
/home/i11/zeitz/projects/tdpot/data/osm_ger/       heavy_live_data interval_min_pot               37.93   
                                                                   lower_bound_cch_pot           128.22   
                                                                   multi_metric_pot              118.11   
                                                                   zero                         1514.35   
                                                   lite_live_data  interval_min_pot               29.97   
                                                                   lower_bound_cch_pot           234.60   
                                                                   multi_metric_pot              171.13   
                                                                   zero                         1530.15   
                                                   predicted       interval_min_pot               22.21   
                                                                   lower_bound_cch_pot           140.43   
                                                                   multi_metric_pot              119.83   
                                                                   zero                         1490.43   
/home/i11/zeitz/projects/tdpot/data/ptv20-eur-car/ live_data       interval_min_pot               25.36   
                                                                   lower_bound_cch_pot           147.72   
                                                                   multi_metric_pot              126.89   
                                                                   zero                         2665.45   
                                                   predicted       interval_min_pot               11.36   
                                                                   lower_bound_cch_pot           101.13   
                                                                   multi_metric_pot               92.47   
                                                                   zero                         2678.24   

                                                                                        num_queue_pops_k  \
graph                                              metric          potential                               
/home/i11/zeitz/projects/tdpot/data/osm_ger/       heavy_live_data interval_min_pot                11.46   
                                                                   lower_bound_cch_pot             89.55   
                                                                   multi_metric_pot                79.47   
                                                                   zero                          1671.60   
                                                   lite_live_data  interval_min_pot                 7.63   
                                                                   lower_bound_cch_pot            158.62   
                                                                   multi_metric_pot               110.76   
                                                                   zero                          1672.51   
                                                   predicted       interval_min_pot                 5.06   
                                                                   lower_bound_cch_pot             92.47   
                                                                   multi_metric_pot                75.11   
                                                                   zero                          1672.64   
/home/i11/zeitz/projects/tdpot/data/ptv20-eur-car/ live_data       interval_min_pot                 7.35   
                                                                   lower_bound_cch_pot            100.38   
     

In [9]:
queries.query('~topo').groupby(['graph', 'metric', 'potential']).mean()[['running_time_ms', 'num_queue_pops_k']]

running_time_ms  \
graph                                              metric          potential                              
/home/i11/zeitz/projects/tdpot/data/osm_ger/       heavy_live_data interval_min_pot               67.82   
                                                                   lower_bound_cch_pot           284.39   
                                                                   multi_metric_pot              262.22   
                                                                   zero                         3459.12   
                                                   lite_live_data  interval_min_pot               49.79   
                                                                   lower_bound_cch_pot           534.63   
                                                                   multi_metric_pot              388.78   
                                                                   zero                         3491.69   
                                                   predicted       interval_min_pot               34.37   
                                                                   lower_bound_cch_pot           313.29   
                                                                   multi_metric_pot              270.16   
                                                                   zero                         3451.78   
/home/i11/zeitz/projects/tdpot/data/ptv20-eur-car/ live_data       interval_min_pot               38.34   
                                                                   lower_bound_cch_pot           319.73   
                                                                   multi_metric_pot              271.83   
                                                                   zero                         5997.14   
                                                   predicted       interval_min_pot               14.04   
                                                                   lower_bound_cch_pot           211.43   
                                                                   multi_metric_pot              194.48   
                                                                   zero                         5926.36   

                                                                                        num_queue_pops_k  
graph                                              metric          potential                              
/home/i11/zeitz/projects/tdpot/data/osm_ger/       heavy_live_data interval_min_pot                72.81  
                                                                   lower_bound_cch_pot            485.74  
                                                                   multi_metric_pot               433.24  
                                                                   zero                          8075.76  
                                                   lite_live_data  interval_min_pot                51.16  
                                                                   lower_bound_cch_pot            827.36  
                                                                   multi_metric_pot               585.03  
                                                                   zero                          8075.22  
                                                   predicted       interval_min_pot                35.71  
                                                                   lower_bound_cch_pot            491.63  
                                                                   multi_metric_pot               403.57  
                                                                   zero                          8079.14  
/home/i11/zeitz/projects/tdpot/data/ptv20-eur-car/ live_data       interval_min_pot                38.19  
                                                                   lower_bound_cch_pot            494.98  
                     

In [10]:
baseline = queries.query('~topo').groupby(['potential', 'graph', 'metric']).mean()['running_time_ms']['zero']

In [15]:
table = queries.query('topo') \
    .groupby(['graph', 'metric', 'potential']) \
    .mean()[['running_time_ms', 'num_queue_pops_k', 'increase_percent']] \
    .reset_index(level=2) \
    .merge(baseline, how='left', suffixes=['', '_baseline'], left_index=True, right_index=True) \
    .reset_index() \
    .set_index(['potential', 'graph', 'metric']) \
    .sort_index() \
    .loc[['lower_bound_cch_pot', 'multi_metric_pot', 'interval_min_pot']]

table['speedup'] = table['running_time_ms_baseline'] / table['running_time_ms']


pre_merged = prepros[['graph', 'potential', 'preprocessing', 'build']].fillna(0.0).groupby(['graph', 'potential']).mean()[['preprocessing', 'build']].reset_index() \
    .merge(cch_ordering.groupby('graph').mean()['cch_ordering_running_time_s'], how='left', on='graph')
pre_merged['total_preprocessing_time_s'] = pre_merged['cch_ordering_running_time_s'] + pre_merged['preprocessing'] / 1000 + pre_merged['build'] / 1000
table = table.reset_index().merge(pre_merged[['graph', 'potential', 'total_preprocessing_time_s']], how='left', on=['graph', 'potential'])
table = table.merge(customs.groupby(['graph', 'metric', 'potential']).mean()[['customization']] / 1000, how='left', on=['graph', 'metric', 'potential'])

table['graph'] = table['graph'].map({ '/home/i11/zeitz/projects/tdpot/data/osm_ger/': 'OSM Ger', '/home/i11/zeitz/projects/tdpot/data/ptv20-eur-car/': 'PTV Eur' })

table = table.set_index(['potential', 'graph', 'metric'])

table = table[['running_time_ms', 'num_queue_pops_k', 'increase_percent', 'speedup', 'total_preprocessing_time_s', 'customization']]
print(table.to_latex())
table

\begin{tabular}{lllrrrrrr}
\toprule
                 &         &           &  running\_time\_ms &  num\_queue\_pops\_k &  increase\_percent &  speedup &  total\_preprocessing\_time\_s &  customization \\
potential & graph & metric &                  &                   &                   &          &                             &                \\
\midrule
lower\_bound\_cch\_pot & OSM Ger & heavy\_live\_data &           128.22 &             89.55 &             19.07 &    26.98 &                      165.24 &            NaN \\
                 &         & lite\_live\_data &           234.60 &            158.62 &             18.96 &    14.88 &                      165.24 &            NaN \\
                 &         & predicted &           140.43 &             92.47 &             12.21 &    24.58 &                      165.24 &            NaN \\
                 & PTV Eur & live\_data &           147.72 &            100.38 &              8.46 &    40.60 &                      249.71 & 

/tmp/ipykernel_42700/1986811400.py:25: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(table.to_latex())


running_time_ms  \
potential           graph   metric                             
lower_bound_cch_pot OSM Ger heavy_live_data           128.22   
                            lite_live_data            234.60   
                            predicted                 140.43   
                    PTV Eur live_data                 147.72   
                            predicted                 101.13   
multi_metric_pot    OSM Ger heavy_live_data           118.11   
                            lite_live_data            171.13   
                            predicted                 119.83   
                    PTV Eur live_data                 126.89   
                            predicted                  92.47   
interval_min_pot    OSM Ger heavy_live_data            37.93   
                            lite_live_data             29.97   
                            predicted                  22.21   
                    PTV Eur live_data                  25.36   
                            predicted                  11.36   

                                             num_queue_pops_k  \
potential           graph   metric                              
lower_bound_cch_pot OSM Ger heavy_live_data             89.55   
                            lite_live_data             158.62   
                            predicted                   92.47   
                    PTV Eur live_data                  100.38   
                            predicted                   63.81   
multi_metric_pot    OSM Ger heavy_live_data             79.47   
                            lite_live_data             110.76   
                            predicted                   75.11   
                    PTV Eur live_data                   82.63   
                            predicted                   57.22   
interval_min_pot    OSM Ger heavy_live_data             11.46   
                            lite_live_data               7.63   
                            predicted                    5.06   
                    PTV Eur live_data                    7.35   
                            predicted                    1.72   

                                             increase_percent  speedup  \
potential           graph   metric                                       
lower_bound_cch_pot OSM Ger heavy_live_data             19.07    26.98   
                            lite_live_data              18.96    14.88   
                            predicted                   12.21    24.58   
                    PTV Eur live_data                    8.46    40.60   
                            predicted                    4.17    58.60   
multi_metric_pot    OSM Ger heavy_live_data             15.81    29.29   
                            lite_live_data              13.01    20.40   
                            predicted                    9.95    28.80   
                    PTV Eur live_data                    5.90    47.26   
                            predicted                    3.46    64.09   
interval_min_pot    OSM Ger heavy_live_data              4.19    91.21   
                            lite_live_data               2.60   116.50   
                            predicted                    1.77   155.42   
                    PTV Eur live_data                    1.73   236.44   
                            predicted                    0.36   521.80   

                                             total_preprocessing_time_s  \
potential           graph   metric                                        
lower_bound_cch_pot OSM Ger heavy_live_data                      165.24   
                            lite_live_data                       165.24   
                            predicted                            165.24   
                    PTV Eur live_data                            249.71   
                            predicted                            249.71   
multi_metric_pot    OSM Ger heavy_live_data                      3